In [1]:
import numpy as np
import pandas as pd
import tables as tb
import zipfile
import os
import glob
import h5py
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler

from bokeh.plotting import figure
from bokeh.io import show, output_notebook

import config as conf
from data import NASDAQ_Data

output_notebook()

Loading BokehJS ...

In [4]:
if not os.path.exists('data'):
    with zipfile.ZipFile('archive.zip', 'r') as zip_ref:
        zip_ref.extractall('data')
    os.remove('archive.zip')
else:
    print('Exists')

Exists


In [2]:
data = NASDAQ_Data()

spy = data.get_ticker('SPY')
print(spy.head())

p = figure(title = 'Spy log returns')
p.line(spy.index, spy['Normalized'], color = 'red')
#p.line(spy.index, spy['Hawkes0.1'], color = 'blue')
#p.line(spy.index, spy.MINMAX, color = 'green')
show(p)

                  Open        High         Low       Close     Volume  \
Date                                                                    
2010-01-04  112.370003  113.389999  111.510002  113.330002  118944600   
2010-01-05  113.260002  113.680000  112.849998  113.629997  111579900   
2010-01-06  113.519997  113.989998  113.430000  113.709999  116074400   
2010-01-07  113.500000  114.330002  113.180000  114.190002  131091100   
2010-01-08  113.889999  114.620003  113.660004  114.570000  126402800   

               ATR14  Normalized    MINMAX  PCT_CHANGE  
Date                                                    
2010-01-04  0.875714    0.003693  0.237083    0.016960  
2010-01-05  0.907143   -0.000366  0.238100    0.002647  
2010-01-06  0.901428   -0.001389  0.238371    0.000704  
2010-01-07  0.970000    0.000770  0.239999    0.004221  
2010-01-08  1.038572    0.000096  0.241287    0.003328  


In [3]:
data.create_dataset(filename = '0.2Thresh_NewFeat.h5')

 37%|███▋      | 2988/8049 [00:13<00:23, 213.74it/s]/Users/mgurumurthi/Projects/DSCI 403 Notebooks/Semester Project/data.py:43: RuntimeWarning: invalid value encountered in divide
  return (X - X.min(axis=0)) / (X.max(axis=0) - X.min(axis=0))
/Users/mgurumurthi/Projects/DSCI 403 Notebooks/Semester Project/data.py:43: RuntimeWarning: invalid value encountered in divide
  return (X - X.min(axis=0)) / (X.max(axis=0) - X.min(axis=0))
 78%|███████▊  | 6311/8049 [00:28<00:08, 213.82it/s]/Users/mgurumurthi/Projects/DSCI 403 Notebooks/Semester Project/data.py:43: RuntimeWarning: invalid value encountered in divide
  return (X - X.min(axis=0)) / (X.max(axis=0) - X.min(axis=0))
/Users/mgurumurthi/Projects/DSCI 403 Notebooks/Semester Project/data.py:43: RuntimeWarning: invalid value encountered in divide
  return (X - X.min(axis=0)) / (X.max(axis=0) - X.min(axis=0))
100%|██████████| 8049/8049 [00:36<00:00, 218.72it/s]

Class Totals : [308, 308, 308]
Didn't Pricess - 
['A', 'AA', 'AAAU', 'AACG', 'AADR', 'AAL', 'AAMC', 'AAME', 'AAN', 'AAOI', 'AAON', 'AAP', 'AAPL', 'AAT', 'AAU', 'AAWW', 'AAXJ', 'AAXN', 'AB', 'ABB', 'ABBV', 'ABC', 'ABCB', 'ABEQ', 'ABEV', 'ABG', 'ABM', 'ABMD', 'ABR', 'ABT', 'ABTX', 'ABUS', 'AC', 'ACA', 'ACAD', 'ACAM', 'ACAMU', 'ACBI', 'ACC', 'ACCO', 'ACEL', 'ACER', 'ACES', 'ACGL', 'ACGLO', 'ACGLP', 'ACH', 'ACHC', 'ACHV', 'ACIA', 'ACIO', 'ACIU', 'ACIW', 'ACLS', 'ACM', 'ACMR', 'ACN', 'ACNB', 'ACOR', 'ACP', 'ACRE', 'ACRS', 'ACRX', 'ACSG', 'ACSI', 'ACST', 'ACT', 'ACTG', 'ACTT', 'ACTTU', 'ACU', 'ACV', 'ACWF', 'ACWI', 'ACWV', 'ACWX', 'ACY', 'ADAP', 'ADBE', 'ADC', 'ADI', 'ADIL', 'ADM', 'ADMA', 'ADME', 'ADMS', 'ADNT', 'ADP', 'ADPT', 'ADRE', 'ADRO', 'ADS', 'ADSK', 'ADSW', 'ADT', 'ADTN', 'ADUS', 'ADVM', 'ADX', 'ADXN', 'AE', 'AEB', 'AEE', 'AEF', 'AEFC', 'AEG', 'AEGN', 'AEIS', 'AEL', 'AEM', 'AEMD', 'AEO', 'AEP', 'AER', 'AERI', 'AES', 'AESE', 'AESR', 'AEY', 'AEYE', 'AFB', 'AFC', 'AFG', 'AFGC', 'AFGH',

In [5]:
with h5py.File('0.2Thresh_NewFeat.h5', 'r') as hf:
    X = np.array(hf['Features'])
    y = np.array(hf['Labels'])

print(X.shape, y.shape)

(36915, 200, 4) (36915, 3)


In [ ]:
class DataGenerator:
    def __init__(self, file):
        self.file = file

    def __call__(self):
        with h5py.File(self.file, 'r') as hf:
            for sample, label in zip(hf["Features"], hf["Labels"]):
                yield sample, label
                
dataset = tf.data.Dataset.from_generator(
    DataGenerator('data.h5'),
    output_signature=(
        tf.TensorSpec(shape=(200, 5), dtype=tf.float64),
        tf.TensorSpec(shape=(3), dtype=tf.int8)
    )
)

In [5]:
with h5py.File('0.2Thresh_NewFeat.h5', 'r') as hf:
    X = np.array(hf['Features'])
    y = np.array(hf['Labels'])

for i in range(X.shape[0]):
    if np.isnan(X[i]).any():
        print(f"{i} NaN present")
    if np.isinf(X[i]).any():
        print(f"{i} NaN present")

In [ ]:
model_path = 'output/RNN_1/Trained_Model/optimized_model.h5'
model = tf.keras.models.load_model(model_path)
model.summary()